# Image Classifier using Transfer Learning

If using colab or other online clould services. add the location of drive where dataset has been stored

In [ ]:
import torch
from torchvision import datasets, models, transforms
import os
import numpy as np
import torch.optim as optim
import time
from torch.utils.data import Dataset, Subset
import copy
import torchvision

## Define Model for Classification

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

class Network(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.model = models.resnet18(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False
        numfts = self.model.fc.in_features
        #----------RESNET - 50
        #self.model.fc = nn.Sequential(nn.Linear(self.model_in_fetures,1000),nn.ReLU(),nn.Dropout(0.5),
        #      nn.Linear(1000,250),nn.ReLU(),nn.Dropout(0.5),
        #      nn.Linear(250,50),nn.ReLU(),nn.Dropout(0.5),
        #      nn.Linear(50,8))
        self.model.fc = nn.Linear(numfts,8)
        #----------RESNET - 18
        #self.model.fc = nn.Sequential(nn.Linear(numfts,250),nn.ReLU(),nn.Dropout(0.5),
        #                              nn.Linear(250,100),nn.ReLU(),nn.Dropout(0.5),
        #                              nn.Linear(100,50),nn.ReLU(),nn.Dropout(0.5),
        #                              nn.Linear(50,8))

    def forward(self, x):
        return  F.log_softmax(self.model(x),dim = 1)
        
    
    def save_model(self):
        
        torch.save(self.state_dict(), 'model_save')


## Tansformation used for Training Dataset

In [ ]:
trans = transforms.Compose([transforms.RandomRotation(25),
                              transforms.RandomResizedCrop(224),
                              transforms.RandomHorizontalFlip(),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

transNoAugment = transforms.Compose([transforms.Resize(255), 
                                  transforms.CenterCrop(224),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


class MyLazyDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        if self.transform:
            x = self.transform(dataset[index][0])
        else:
            x = dataset[index][0]
        y = dataset[index][1]
        return x, y
    
    def __len__(self):
        return len(dataset)
    
# Load entire dataset once
data_dir = '----add---location----'
dataset = datasets.ImageFolder(data_dir)

traindataset = MyLazyDataset(dataset,trans)
valdataset = MyLazyDataset(dataset,transNoAugment)
testdataset = MyLazyDataset(dataset,transNoAugment)

### Test-Train Split as well as Data Loders

In [ ]:
train_size = 0.8
num_train = len(traindataset)
indices = list(range(num_train))
split = int(np.floor(train_size * num_train))
split2 = int(np.floor((train_size+(1-train_size)/2) * num_train))
np.random.shuffle(indices)
train_idx, valid_idx, test_idx = indices[:split], indices[split:split2], indices[split2:]

traindata = Subset(traindataset, indices=train_idx)
valdata = Subset(valdataset, indices=valid_idx)
testdata = Subset(testdataset, indices=test_idx)

num_workers = 5
batch_size = 8

if __name__ == '__main__':
    trainLoader = torch.utils.data.DataLoader(traindata, batch_size=batch_size, 
                                              num_workers=num_workers, drop_last=True)
    valLoader = torch.utils.data.DataLoader(valdata, batch_size=batch_size, 
                                            num_workers=num_workers, drop_last=True)
    testLoader = torch.utils.data.DataLoader(testdata, batch_size=batch_size,
                                             num_workers=num_workers, drop_last=True)

### Visualisation of Images

In [ ]:
import matplotlib.pyplot as plt

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(trainLoader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out)

### Send Network to GPU(if available)

In [ ]:
net = Network()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

### Define Loss Function

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
#optimizer = torch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

### Function for training

In [ ]:
def train_model(model,numEpoch,criterion,optimizer,scheduler):
    since = time.time()
    best_acc = 0

    for epoch in range(numEpoch):  # loop over the dataset multiple times
        #valcount = 0
        running_loss = 0.0
        train_total = 0.0
        train_correct = 0.0
        for data in trainLoader:
            model.train()
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device),data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_total += labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_correct += torch.sum(predicted == labels.data)
            running_loss += loss.item() * inputs.size(0)

            scheduler.step()
            # print statistics
            epoch_acc_train = 100 * train_correct / train_total
            epoch_loss = running_loss/train_total
            print("train total [%d] " %(train_total))
            print('[%d] epoch loss : %.3f' %(epoch + 1,epoch_loss))
            print('[%d] train accuracy: %.3f' %(epoch + 1,epoch_acc_train))

      #####Validation Step
        val_correct = 0.0
        val_total = 0.0
        with torch.no_grad():
            model.eval()
            for data in valLoader:
                images, labels = data[0].to(device),data[1].to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        epoch_acc_val = 100 * val_correct / val_total
        print("val total [%d] " %(val_total))
        print('[%d] val accuracy: %.3f' %(epoch + 1,epoch_acc_val))
        if(epoch_acc_val > best_acc):
            best_model_wts = copy.deepcopy(model.state_dict())
            print('[%d] Increased Epoch Accuracy: %.3f' %
                    (epoch + 1, epoch_acc_val))
            best_acc = epoch_acc_val

        #if(currentValAccuracy > prevValidationAccuracy):
        #  print("Current Validation is more than previous for {0} pass : {1}".format(valcount+1,currentValAccuracy))
        #  valcount += 1
        #  if(valcount > 3):
        #    break

    endtime = time.time()
    print('Time Elapsed in mins - {0}'.format((endtime-since)/60))
    print('Finished Training')
    model.load_state_dict(best_model_wts)
    return model

## Train the Model

In [ ]:
epoch = 100
trained_model = train_model(net,epoch,criterion,optimizer,exp_lr_scheduler)

### Saving Model and Load preious model

In [ ]:
#torch.save(trained_model,'model100_epoch_resnet18_1linear')
trained_model = torch.load('model100_epoch_resnet18_1linear')

### Inference on test dataset

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  trained_model.eval()
  for data in testLoader:
      images, labels = data[0].to(device),data[1].to(device)
      outputs = trained_model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

print('Accuracy of the network on test dataset is : %d %%' % (
    100 * correct / total))

### Storing and saving result in CSV

In [ ]:
import glob
import re
from PIL import Image
from torch.autograd import Variable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#net = model.Network().to(device)
#net.load_state_dict(torch.load('model'))
net = trained_model

net.eval()
filelist = glob.glob('----File_Location---/*.png')
loader = transforms.Compose([transforms.Resize(224),transforms.ToTensor()])
prediction = torch.tensor([],dtype = torch.float).to(device)
i = 0
label = []

for fname in filelist:
    image = Image.open(fname).convert('RGB')
    image = loader(image).to(device)
    
    image = Variable(image)
    image = image.unsqueeze(0)
    pred_image = net(image)
    result_image = pred_image.argmax(dim = 1)
        
    prediction = torch.cat((prediction,result_image),dim = 0)
    prediction = prediction.int()
    fname2 = re.findall(r'\d+',fname)
    fname2 = list(map(int,fname2))
    label.append(fname2[0])

In [ ]:
#print(predten.argmax(dim = 1))
class_names_predicted  = []
prediction = prediction.to('cpu').numpy()
class_names = dataset.classes
label1 = label
for i in prediction:
    class_names_predicted.append(class_names[i])


In [ ]:
import pandas as pd
data_val = pd.DataFrame({'Id':label1,'Category':prediction,'Predicted class Label':class_names_predicted})
data_val = data_val.sort_values('Id')
data_val.to_csv('test2.csv',index = False)

In [ ]:
data_val